In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# 그래프 기본 설정
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False
import matplotlib as mpl
import matplotlib.font_manager as fm

# 폰트 파일 경로 지정
font_path = '/content/drive/MyDrive/font/NanumGothic.ttf'

# 폰트 등록 및 matplotlib 설정
font_name = fm.FontProperties(fname=font_path).get_name()
fm.fontManager.addfont(font_path)
mpl.rc('font', family=font_name)

In [ ]:
import pandas as pd
from google.colab import drive, files

# 1. Google Drive 마운트
drive.mount('/content/drive')

# 2. 파일 경로 설정 (사용자 지정 경로)
data_root = '/content/drive/MyDrive/Colab Notebooks/final_project/final_sampling'
file_path = f'{data_root}/train_12.parquet'

# # 3. Parquet 파일 불러오기
# df = pd.read_parquet(file_path)

# # 4. CSV로 저장 (엑셀용 UTF-8-sig 인코딩)
# csv_filename = 'train_12.csv'
# df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

# # 5. 로컬로 다운로드
# files.download(csv_filename)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 파일 경로 설정
data_root = '/content/drive/MyDrive/Colab Notebooks/final_project/final_sampling'

X_path = f'{data_root}/train_12.parquet'
y_path = f'{data_root}/segment_12.parquet'

# 데이터 불러오기
X_df = pd.read_parquet(X_path)
y_df = pd.read_parquet(y_path)

# ID 기준으로 병합
df = pd.merge(X_df, y_df, on='ID', how='inner')

# 확인
print(df.shape)
df.head()

(400000, 313)


,기준년월,ID,이용금액_R3M_신용체크,입회경과개월수_신용,_1순위카드이용금액,회원여부_이용가능_카드론,이용거절여부_카드론,최종카드발급경과월,이용금액_R3M_신용,_1순위카드이용건수,...,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M,잔액_한도소진율,증감율_카드론_분기,증감율_일시불_분기,증감율_체크_분기,증감율_카드론_전월,변동률_RV평잔,Segment
0,201812,TRAIN_000000,-454,71,3027,0,0,27,-454,25,...,0.000000,0.878859,1.398627,3.407027,0.143423,-0.609648,0.086480,0.048726,0.321733,D
1,201812,TRAIN_000001,7089,16,7259,1,0,23,7089,31,...,0.000000,0.000000,0.000000,5.196375,0.143423,-0.597498,0.086480,0.048726,0.214949,E
2,201812,TRAIN_000002,27336,128,26968,0,0,25,27336,52,...,-0.115879,0.187467,-1.198788,4.332080,0.143423,-0.154887,0.086480,0.048726,-2.839702,C
3,201812,TRAIN_000003,4270,31,4807,0,0,22,4270,27,...,0.000000,0.781401,1.282494,6.248569,0.143423,0.593422,0.086480,0.048726,0.321733,D
4,201812,TRAIN_000004,9385,6,3989,1,0,20,7387,-2,...,0.000000,0.762016,0.986860,-2.056257,0.143423,4.964378,-0.654283,0.048726,0.321733,E


In [ ]:
# 타겟 변수와 피처 나누기
target_col = 'Segment'
X = df.drop(columns=[target_col])
y = df[target_col]

In [ ]:
# Label Encoder
from sklearn.preprocessing import LabelEncoder

# 타겟 값 y 인코딩
le_y = LabelEncoder()
y = le_y.fit_transform(y)

# 타겟 값 X 인코딩
cat_cols = X.select_dtypes(include='object').columns
le = LabelEncoder()
for col in cat_cols:
    X[col] = le.fit_transform(X[col].astype(str))

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

전처리 완료 변수들
X, y: 병합 및 인코딩된 전체 데이터셋
le_y: y(LabelEncoder 객체)
X_train_scaled, X_test_scaled, y_train, y_test: 스케일링된 train/test 데이터셋

# Segment 'E' vs 나머지 (XGBoost)

In [ ]:
from xgboost import XGBClassifier
import numpy as np

# Step 1: E vs Others 분류용 라벨 생성
y_train_step1 = (y_train == le_y.transform(['E'])[0]).astype(int)
y_test_step1 = (y_test == le_y.transform(['E'])[0]).astype(int)

model_step1 = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
model_step1.fit(X_train_scaled, y_train_step1)

# 예측: 전체 test set에 대해 E 확률
prob_E = model_step1.predict_proba(X_test_scaled)[:, 1]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:40:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


# Segment 'C' vs 'D' vs 나머지 (RandomForest)
- 간단 분류라 랜덤포레스트. but 추후 성능 향상 가능할 것 같다면 XGBoost도 시도

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Step 2 대상 추출: model1이 E가 아니라고 판단한 인덱스
not_E_indices = prob_E < 0.5
X_test_step2 = X_test_scaled[not_E_indices]
y_test_not_E = y_test[not_E_indices]

# Step 2 라벨: C=0, D=1, Others(A/B)=2
y_train_step2 = np.where(
    y_train == le_y.transform(['C'])[0], 0,
    np.where(y_train == le_y.transform(['D'])[0], 1, 2)
)
y_test_step2 = np.where(
    y_test_not_E == le_y.transform(['C'])[0], 0,
    np.where(y_test_not_E == le_y.transform(['D'])[0], 1, 2)
)

model_step2 = RandomForestClassifier(random_state=42)
model_step2.fit(X_train_scaled, y_train_step2)

# 예측
pred_step2 = model_step2.predict(X_test_step2)

# Segment 'A' vs 'B'(XGBoost)

In [ ]:
# Step 3 대상 추출: model2가 Others라고 판단한 인덱스
from sklearn.utils import resample

step2_is_ab = pred_step2 == 2
X_test_step3 = X_test_step2[step2_is_ab]
y_test_step3_true = y_test_not_E[step2_is_ab]

# Step 3 학습 라벨: A=0, B=1 (학습 대상도 A/B만 뽑아야 함)
ab_mask = np.isin(y_train, le_y.transform(['A', 'B']))
X_train_ab = X_train_scaled[ab_mask]
y_train_ab = np.where(y_train[ab_mask] == le_y.transform(['A'])[0], 0, 1)

model_step3 = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
model_step3.fit(X_train_ab, y_train_ab)

# 예측
pred_step3 = model_step3.predict(X_test_step3)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:49:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


# 최종 결과 결합

In [ ]:
# 기본값: 전부 E라고 초기화
final_preds = np.full_like(y_test, fill_value=le_y.transform(['E'])[0])

# Step 1에서 Not E로 분류된 인덱스 → Step 2 결과 대입
final_preds[not_E_indices] = np.where(pred_step2 == 0, le_y.transform(['C'])[0],
                              np.where(pred_step2 == 1, le_y.transform(['D'])[0], -1))

# Step 2에서 A/B로 분류된 것만 Step 3 예측으로 대체
final_preds_idx_step3 = np.where(not_E_indices)[0][step2_is_ab]
final_preds[final_preds_idx_step3] = np.where(pred_step3 == 0, le_y.transform(['A'])[0], le_y.transform(['B'])[0])

#최종 평가

In [ ]:
from sklearn.metrics import classification_report

print("Final Classification Report:\n",
      classification_report(y_test, final_preds, target_names=le_y.classes_))

Final Classification Report:
               precision    recall  f1-score   support

           A       0.00      0.03      0.00        32
           B       0.00      0.00      0.00         5
           C       0.71      0.51      0.59      4253
           D       0.72      0.55      0.62     11642
           E       0.95      0.97      0.96     64068

    accuracy                           0.88     80000
   macro avg       0.48      0.41      0.44     80000
weighted avg       0.91      0.88      0.89     80000



- A, B 클래스는 여전히 거의 예측 불가

| Segment | Precision | Recall | F1-score | 해석                        |
| ------- | --------- | ------ | -------- | ------------------------- |
| **E**   | 0.95      | 0.97   | 0.96     | 거의 완벽하게 잘 맞춤 (데이터가 가장 많음) |
| **D**   | 0.72      | 0.55   | 0.62     | 꽤 잘 예측하고 있음 (다음으로 많은 클래스) |
| **C**   | 0.71      | 0.51   | 0.59     | 비교적 성능 양호함                |
| **A**   | 0.00      | 0.03   | 0.00     | 완전 실패에 가까움 (소수 클래스)       |
| **B**   | 0.00      | 0.00   | 0.00     | 예측 거의 안 됨                 |

- 방법 3은 E, C, D 예측을 잘 다듬는 데는 효과적이나 데이터 구조적 한계 여전

#어카지 1. A, B 클래스 EDA를 다시
<분석 목표>
- A와 B 클래스는 어떤 특성이 있는가?
- 다른 세그먼트 대비 구분 가능한 특징이 있는가?
- 모델링 시 어떤 변수들이 도움이 될까?
- SMOTE나 별도 모델링 대상이 될만한가?

<인사이트>
- A/B가 심하게 적음 → SMOTE나 별도 모델 필수
- 총이용건수/금액 등은 B가 거의 0에 가까운 반면, A는 넓게 퍼져 있음
- 할부이용건수, 대중교통이용건수 등은 A에 비해 B에서 극단적으로 낮거나 아예 없음 -> A/B가 확연히 다른 컬럼

#어카지2. SMOTE + XGB/RdForest
단일 모델 각 돌려보고 A/B 구분을 위한 전용 모델링

In [ ]:
# 필요한 라이브러리
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
import numpy as np

# 데이터 불러오기
X = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/final_project/final_sampling/train_12.parquet')
y = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/final_project/final_sampling/segment_12.parquet')

# 병합
df = pd.merge(X, y, on='ID', how='inner')
df['Segment'] = LabelEncoder().fit_transform(df['Segment'])

# 피처/라벨 분리
X = df.drop(columns=['Segment'])
y = df['Segment']

# 범주형 인코딩
cat_cols = X.select_dtypes(include='object').columns
for col in cat_cols:
    X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SMOTE: A=0, B=1 → 각 2000개로 증강
smote = SMOTE(sampling_strategy={0: 2000, 1: 2000}, random_state=42, k_neighbors=3)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# 클래스 분포 확인
print("Before SMOTE:", np.bincount(y_train))
print("After SMOTE:", np.bincount(y_train_resampled))

Before SMOTE: [   130     19  17012  46565 256274]
After SMOTE: [  2000   2000  17012  46565 256274]


In [ ]:
!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import numpy as np

# 1. 데이터 불러오기
X_df = pd.read_parquet("/content/drive/MyDrive/Colab Notebooks/final_project/final_sampling/train_12.parquet")
y_df = pd.read_parquet("/content/drive/MyDrive/Colab Notebooks/final_project/final_sampling/segment_12.parquet")
df = pd.merge(X_df, y_df, on='ID', how='inner')

# 2. 인코딩
le_y = LabelEncoder()
df['Segment'] = le_y.fit_transform(df['Segment'])
X = df.drop(columns=['Segment'])
y = df['Segment']

cat_cols = X.select_dtypes(include='object').columns
for col in cat_cols:
    X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# 3. 분할 + 스케일링
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. SMOTE (A=0, B=1만 증강)
smote = SMOTE(sampling_strategy={0: 2000, 1: 2000}, random_state=42, k_neighbors=3)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# 5. XGBoost
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(X_train_resampled, y_train_resampled)
y_pred_xgb = xgb.predict(X_test_scaled)

# 6. RandomForest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_resampled, y_train_resampled)
y_pred_rf = rf.predict(X_test_scaled)

# 7. 결과 출력
print("XGBoost:\n", classification_report(y_test, y_pred_xgb, target_names=le_y.classes_))
print("RandomForest:\n", classification_report(y_test, y_pred_rf, target_names=le_y.classes_))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:50:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost:
               precision    recall  f1-score   support

           A       0.44      0.22      0.29        32
           B       0.00      0.00      0.00         5
           C       0.72      0.60      0.65      4253
           D       0.74      0.69      0.71     11642
           E       0.95      0.97      0.96     64068

    accuracy                           0.91     80000
   macro avg       0.57      0.50      0.52     80000
weighted avg       0.91      0.91      0.91     80000

RandomForest:
               precision    recall  f1-score   support

           A       0.62      0.16      0.25        32
           B       0.00      0.00      0.00         5
           C       0.71      0.51      0.59      4253
           D       0.68      0.58      0.62     11642
           E       0.93      0.97      0.95     64068

    accuracy                           0.89     80000
   macro avg       0.59      0.44      0.48     80000
weighted avg       0.88      0.89      0.88     8000

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


A, B 클래스 샘플 수 조정해서 다시

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

# 데이터 불러오기 및 병합
X_df = pd.read_parquet(f'{data_root}/train_12.parquet')
y_df = pd.read_parquet(f'{data_root}/segment_12.parquet')
df = pd.merge(X_df, y_df, on='ID', how='inner')

f'{data_root}/train_12.parquet'

# 타겟 인코딩
le_y = LabelEncoder()
df['Segment'] = le_y.fit_transform(df['Segment'])
X = df.drop(columns=['Segment'])
y = df['Segment']

# 범주형 인코딩
cat_cols = X.select_dtypes(include='object').columns
for col in cat_cols:
    X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# 데이터 분할 및 스케일링
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SMOTE: A → 3000, B → 5000
smote = SMOTE(sampling_strategy={0: 3000, 1: 5000}, random_state=42, k_neighbors=3)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# 모델 학습
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
rf = RandomForestClassifier(random_state=42)

xgb.fit(X_train_resampled, y_train_resampled)
rf.fit(X_train_resampled, y_train_resampled)

# 예측
y_pred_xgb = xgb.predict(X_test_scaled)
y_pred_rf = rf.predict(X_test_scaled)

# 평가 출력
print("XGBoost Classification Report:\n", classification_report(y_test, y_pred_xgb, target_names=le_y.classes_))
print("\nRandomForest Classification Report:\n", classification_report(y_test, y_pred_rf, target_names=le_y.classes_))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:02:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Classification Report:
               precision    recall  f1-score   support

           A       0.39      0.28      0.33        32
           B       0.00      0.00      0.00         5
           C       0.72      0.59      0.65      4253
           D       0.74      0.69      0.71     11642
           E       0.95      0.97      0.96     64068

    accuracy                           0.91     80000
   macro avg       0.56      0.51      0.53     80000
weighted avg       0.91      0.91      0.91     80000


RandomForest Classification Report:
               precision    recall  f1-score   support

           A       0.33      0.09      0.15        32
           B       0.00      0.00      0.00         5
           C       0.70      0.51      0.59      4253
           D       0.68      0.58      0.62     11642
           E       0.93      0.97      0.95     64068

    accuracy                           0.89     80000
   macro avg       0.53      0.43      0.46     80000
weighte

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#어카지3. A vs B 분류 다시 (XGBoost + SMOTE 1000:2000)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

# 1. 데이터 불러오기 (A/B만 포함된 CSV 파일)
ab_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/final_project/final_sampling/segment_ab.csv")

# 2. 피처/타겟 분리
X_ab = ab_df.drop(columns=["ID", "Segment"])
y_ab = ab_df["Segment"]

# 3. 타겟 라벨 인코딩 (A → 0, B → 1)
le_ab = LabelEncoder()
y_encoded = le_ab.fit_transform(y_ab)

# 4. 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_ab)

# 5. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

# 6. SMOTE 적용 (A: 1000, B: 2000 샘플로 증강)
smote = SMOTE(sampling_strategy={0: 1000, 1: 2000}, random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# 7. XGBoost 모델 학습
model = XGBClassifier(random_state=42, eval_metric="mlogloss")
model.fit(X_train_res, y_train_res)

# 8. 예측 및 평가
y_pred = model.predict(X_test)
target_names = le_ab.classes_  # ['A', 'B']
print("Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=target_names))

Classification Report:

              precision    recall  f1-score   support

           A       0.91      0.94      0.93        33
           B       0.50      0.40      0.44         5

    accuracy                           0.87        38
   macro avg       0.71      0.67      0.68        38
weighted avg       0.86      0.87      0.86        38



- 클래스 A 안정적 분류, B는 여전히 낮지만 이전보다 개선됨
- 이 A/B 분류기는 최종 분류에서 A/B 구간으로 분류된 샘플에만 적용 => 전체 모델 파이프라인에서 마지막 미세조정 역할이기 때문에,
현재 수준의 성능이면 충분히 도입할 수 있음


#어카지4. A vs B Voting / Stacking Classifier
어떤 모델이 A/B 분류에 최적일지

In [ ]:
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

# 1. 기본 분류기들 정의
clf1 = XGBClassifier(random_state=42, eval_metric="mlogloss")
clf2 = RandomForestClassifier(random_state=42)
clf3 = LogisticRegression(max_iter=1000)

# 2. VotingClassifier (하드 보팅)
voting_clf = VotingClassifier(estimators=[
    ('xgb', clf1),
    ('rf', clf2),
    ('lr', clf3)
], voting='hard')

voting_clf.fit(X_train_res, y_train_res)
y_pred_vote = voting_clf.predict(X_test)

print("Voting Classifier Report:")
print(classification_report(y_test, y_pred_vote, target_names=le_ab.classes_))


# 3. StackingClassifier
stacking_clf = StackingClassifier(
    estimators=[
        ('xgb', clf1),
        ('rf', clf2),
        ('lr', clf3)
    ],
    final_estimator=LogisticRegression(max_iter=1000),
    passthrough=False
)

stacking_clf.fit(X_train_res, y_train_res)
y_pred_stack = stacking_clf.predict(X_test)

print("\nStacking Classifier Report:")
print(classification_report(y_test, y_pred_stack, target_names=le_ab.classes_))

Voting Classifier Report:
              precision    recall  f1-score   support

           A       0.91      0.97      0.94        33
           B       0.67      0.40      0.50         5

    accuracy                           0.89        38
   macro avg       0.79      0.68      0.72        38
weighted avg       0.88      0.89      0.88        38


Stacking Classifier Report:
              precision    recall  f1-score   support

           A       0.91      0.97      0.94        33
           B       0.67      0.40      0.50         5

    accuracy                           0.89        38
   macro avg       0.79      0.68      0.72        38
weighted avg       0.88      0.89      0.88        38



- 클래스 A 성능: Precision/Recall/F1 모두 거의 0.94 이상으로 안정적. A는 대부분 잘 분류되고 있음.
- 클래스 B 성능: Recall 0.40 → 5개 중 2개 맞춤
- 이전 SMOTE + 단일모델(XGBoost, RF)보다 정확도, F1 모두 소폭 향상

- Voting vs Stacking 결과가 동일한 이유: A/B 데이터에서 모델 수가 3개이고, 예측 대상이 2개 뿐 → 각 모델의 예측이 서로 비슷하게 나오면 Stacking도 큰 차이 없이 따라감
- 특히 Stacking에서 사용하는 LogisticRegression은
다수 모델이 명확한 우세를 보일 때 Voting과 유사하게 행동할 수 있음

#어카지5. 파이프라인에 적용
++ Soft Voting도 비교해볼까? test 데이터 수 적어서 cross_val_score 기반 평가?

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# [전제]
# - model1: E vs Not E (XGBoost)
# - model2: C/D vs Not C/D (XGBoost or RF)
# - model3: A vs B (VotingClassifier) ← 지금 우리가 만든 최종 모델!

# Step 1: E 여부 예측
pred_step1 = model1.predict(X_test)
is_E = pred_step1 == 1  # 1이면 E

# Step 2: C/D vs A/B 여부 예측 (Step1에서 E가 아닌 샘플만 예측)
X_step2 = X_test[~is_E]
pred_step2 = model2.predict(X_step2)
# pred_step2: 0 = C, 1 = D, 2 = A/B

# Step 3: A/B 모델로 재예측할 인덱스 찾기
step2_is_ab = pred_step2 == 2
X_step3 = X_step2[step2_is_ab]
pred_step3 = model3.predict(X_step3)  # 최종 VotingClassifier

# 최종 결과 조립 (모두 E로 초기화)
final_preds = np.full_like(pred_step1, fill_value=le_y.transform(['E'])[0])

# Step 2: C/D 예측 대입
not_E_indices = np.where(~is_E)[0]
final_preds[not_E_indices] = np.where(pred_step2 == 0, le_y.transform(['C'])[0],
                              np.where(pred_step2 == 1, le_y.transform(['D'])[0], -1))

# Step 3: A/B 예측 대입
final_preds_idx_step3 = not_E_indices[step2_is_ab]
final_preds[final_preds_idx_step3] = np.where(pred_step3 == 0, le_y.transform(['A'])[0], le_y.transform(['B'])[0])

# 최종 평가
print("✅ Final Classification Report:")
print(classification_report(y_test, final_preds, target_names=le_y.classes_))
